In [ ]:
import pyrealsense2 as rs
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
from PIL import Image, ImageTk
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

baseline = 0.05
focal_length = 640
disparity = 50
depth = baseline * focal_length / disparity

depth_thresh = 500

pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

window = tk.Tk()
window.title("Depth Detection")
window.geometry("1280x480")

canvas = tk.Canvas(window, width=1280, height=480)
canvas.pack()

warn = 0

def send_mail(subject, message):
    
    sender_email = "mamleshsurya6@gmail.com" 
    receiver_email = "mjonathanvineeth@gmail.com" 
    password = "smpfnubooeqywwqp"
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    msg.attach(MIMEText(message, 'plain'))

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(sender_email, password)

    server.send_message(msg)
    print("mail sent")
    server.quit()
    
def alert(a):
    if not(a) :
        if warn > 0:
            if warn == 11:
                warn = 10;
            warn -= 1;
            
    else:
        warn += 1
        if warn == 25:
            send_email("Someone is at your porch!","""Hello,
            this is your home security system to inform that someone was spotted on your porch!
            No need to panic, just keep an eye on the camera!""")
        if warn == 100:
            send_email("Anomaly detected","""Hello,
            This is your home security system to inform you regarding the anomaly at your porch.
            There has been unusual activity happening for some time now, we suggest you check it out!""")
        
        

def anomaly_check():
    if (np.sum(mask) / 255.0) > (0.1 * mask.shape[0] * mask.shape[1]):
        contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
        cnts = sorted(contours, key=cv.contourArea, reverse=True)
        if cv.contourArea(cnts[0]) > 0.1 * mask.shape[0] * mask.shape[1]:
            x, y, w, h = cv.boundingRect(cnts[0])

            cv.rectangle(output_canvas, (x, y), (x + w, y + h), (0, 0, 255), 2)

            cv.rectangle(output_canvas, (x, y), (x + w, y + h), (255, 0, 0), 2)

            cx = int(x + w / 2)
            cy = int(y + h / 2)
            cv.drawMarker(output_canvas, (cx + 640, cy), (0, 0, 255), markerType=cv.MARKER_CROSS, thickness=2)
            center_distance = depth_frame.get_distance(cx, cy)

            bottom_center = (output_canvas.shape[1] * 3 // 4, output_canvas.shape[0])
            cv.line(output_canvas, bottom_center, (cx + 640, cy), (0, 0, 255), thickness=2)

            cv.putText(output_canvas, "WARNING !", (cx + 645, cy - 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255),2, cv.LINE_AA)
            cv.putText(output_canvas, "Object at", (cx + 645, cy), cv.FONT_HERSHEY_SIMPLEX, 1, (100, 10, 25), 2,cv.LINE_AA)
            cv.putText(output_canvas, "{:.2f} m".format(center_distance), (x + 5, y + 40),cv.FONT_HERSHEY_SIMPLEX, 1, (100, 10, 25), 2, cv.LINE_AA)
            alert(True)
            
    else:
        cv.putText(output_canvas, "SAFE", (100, 100), cv.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 0), 2, cv.LINE_AA)
        bottom_center = (output_canvas.shape[1] * 3 // 4, output_canvas.shape[0])
        cx = 320
        cy = 240
        color = (0, 255, 0)  
        cv.line(output_canvas, bottom_center, (cx + 640, cy), color, thickness=2)
        alert(False)

def update_image():
    while True:
        time.sleep(0.2)
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        depth_colormap = cv.applyColorMap(cv.convertScaleAbs(depth_image, alpha=0.03), cv.COLOR_GRAY2BGR)

        output_canvas = np.zeros((480, 1280, 3), dtype=np.uint8)
        output_canvas[:, 0:640, :] = color_image
        output_canvas[:, 640:, :] = depth_colormap

        kernel = np.ones((1, 1), np.uint16)
        mask = cv.inRange(depth_image, depth_thresh, 1500)
        mask = cv.dilate(mask, kernel, iterations=2)
        mask = cv.erode(mask, kernel, iterations=1)
        
        anomaly_check()

        img = Image.fromarray(output_canvas)
        imgtk = ImageTk.PhotoImage(image=img)

        canvas.create_image(0, 0, anchor=tk.NW, image=imgtk)
        canvas.image = imgtk

        window.update()

window.mainloop()
update_image()
pipeline.stop()